# Preprocessing Script Check

In this notebook we are goint to chack how the proprocessing has been done. Run First the `Apendix A
`


In [4]:

# Data Science Packages
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from  matplotlib import pyplot
import seaborn as sns
import warnings
# Seaborn Style
sns.set(style='ticks')
sns.set_style({'font.family': 'Hiragino Maru Gothic Pro'})
sns.set_palette("cool")

# Pandas Style
pd.set_option("display.max_column", 9999)
pd.set_option("display.max_row", 9999)

# Ignore annoying warning 
warnings.filterwarnings('ignore')

In [5]:
spark = spark_init()

# log that main ETL job is starting
print('Preprocessing is up-and-running')
cpetypes = ['W724Ci', 'W724Ci_11AC', 'HA35-11']
dataset_version_raw = 'data_w32_g3'
dataset_version_preprocessed = 'Family-all'
root_dir = '../data'

Preprocessing is up-and-running


# 1. Data loading

In [6]:
print('\n' * 1 + '-' * 50 + '\n PREPROCESSING ' + '\n')
print('Dataset Version Raw:', dataset_version_raw)
print('Dataset Version Preprocessed:', dataset_version_preprocessed)
print('cpe types:', [i for i in cpetypes])
print('\n' + '-' * 50)


--------------------------------------------------
 PREPROCESSING 

Dataset Version Raw: data_w32_g3
Dataset Version Preprocessed: Family-all
cpe types: ['W724Ci', 'W724Ci_11AC', 'HA35-11', 'Speedport Plus', 'ZXDSL931VII', 'Speedport Plus181', 'ZXDSL 931VII V4.0']

--------------------------------------------------


In [7]:
# Load Data
path_data_raw = root_dir + '/raw/' + dataset_version_raw
path_data_preprocessed = root_dir + '/preprocessed/' + dataset_version_raw + '/' + dataset_version_preprocessed
path_data_interim = root_dir + '/interim'
df_train = spark.read.parquet(path_data_raw + "/train.parquet")
df_test = spark.read.parquet(path_data_raw + "/test.parquet")

Join the training and test dataset

In [6]:
df = df_train.union(df_test)
df.write.mode('overwrite').parquet(path_data_raw + "/test_train.parquet")
df = spark.read.parquet(path_data_raw + "/test_train.parquet")

In [7]:
df.limit(5).toPandas()

,assetid,CPE_SERIAL_NUMBER,TICKET_ID,ASSET_ID,TICKET_TENTATIVE_ROOT_CAUSE,START_DATE_TICKET,DATE_SCAN_START,DATE_SCAN_END,CPE_ISSUE,WINDOW_END,WINDOW_START,acs_eventid,dslam_distinct_modulation,dslam_no_of_dominant_modulation,dslam_code_of_dominant_modulation,dslam_avg_bitrate_us,dslam_avg_bitrate_ds,dslam_avg_attenuation_ds,dslam_avg_attenuation_us,dslam_avg_power_us,dslam_avg_power_ds,dslam_avg_att_bitrate_us,dslam_avg_att_bitrate_ds,dslam_avg_snr_us,dslam_avg_snr_ds,dslam_no_counts,dslam_avg_bandline_ds,dslam_avg_bandline_us,dslam_sum_cv_us,dslam_count_cv_us,dslam_sum_cv_ds,dslam_count_cv_ds,dslam_sum_es_ds,dslam_count_es_ds,dslam_sum_es_us,dslam_count_es_us,dslam_sum_ses_ds,dslam_count_ses_ds,dslam_sum_ses_us,dslam_count_ses_us,dslam_sum_fec_ds,dslam_count_fec_ds,dslam_sum_fec_us,dslam_count_fec_us,dslam_inits,dslam_port_inst_id,dslam_rg_port_inst_id,dslam_prim_izvod_inst_id,snmp_no_lossoflink,snmp_no_linkup,snmp_no_dyinggasp,acs_xdslline_dslUptime,acs_xdslline_cpeUptime,acs_xdslline_cpuUsagePercentage,acs_xdslline_freeMemoryKb,acs_xdslline_totalMemoryKb,acs_xdslline_usedMemoryKb,acs_xdslline_natCurrent,acs_xdslline_natMax,acs_xdslline_crc,acs_xdslline_fec,acs_xdslline_hec,acs_xdslline_cellDelin,acs_xdslline_erroredSecs,acs_xdslline_initErrors,acs_xdslline_initTimeOuts,acs_xdslline_severelyErroredSecs,acs_xdslline_linkRetrain,acs_xdslline_lossOfFraming,acs_xdslline_lossOfPower,acs_xdslline_transmitBlocks,acs_xdslline_receiveBlocks,acs_xdslline_downAttenuationDb,acs_xdslline_downCurrRateKbps,acs_xdslline_downMaxRateKbps,acs_xdslline_downNoiseMarginDb,acs_xdslline_downPowerDbm,acs_xdslline_upAttenuationDb,acs_xdslline_upCurrRateKbps,acs_xdslline_upMaxRateKbps,acs_xdslline_upNoiseMarginDb,acs_xdslline_upPowerDbm,acs_xdslline_atuccrc,acs_xdslline_atucfec,acs_xdslline_atuchec,datum,dslam_date_inserted,dslam_platforma,dslam_model,dslam_card_type,dslam_status_porta,dslam_bandwidth,dslam_vdsl_adsl,dslam_vendor,dslam_status,dslam_internet,dslam_iptv,dslam_voip,dslam_regija,acs_@type,acs_status,acs_description,acs_crm_accessNetwork,acs_crm_dslamFrame,acs_crm_dslamSlot,acs_crm_serviceName,acs_xdslline_manufacturer,acs_xdslline_manufacturerOui,acs_xdslline_softwareVersion,acs_xdslline_additionalSoftwareVersion,acs_xdslline_anexType,acs_xdslline_dslType,acs_xdslline_linkEncapsulationType,acs_xdslline_pppoEConnectionStatus,acs_xdslline_pppoELastConnectionError,acs_xdslline_pppoENATEnabled,acs_xdslline_pppoEUsername,acs_createdmsec,cpetype,lb_id,lb_networkdeviceid,lb_eventdate,lb_eventtime,lb_bssid,lb_channel,lb_channelsinuse,lb_transmitpower,lb_min_rssi_per_cpe,lb_max_rssi_per_cpe,lb_avg_rssi_per_cpe,lb_perc5_rssi_per_cpe,lb_perc25_rssi_per_cpe,lb_perc50_rssi_per_cpe,lb_perc75_rssi_per_cpe,lb_perc95_rssi_per_cpe,lb_bytesreceived,lb_bytessent,lb_errorsreceived,lb_errorssent,lb_packetsreceived,lb_packetssent,lb_bytesreceiveddelta,lb_bytessentdelta,lb_errorsreceiveddelta,lb_errorssentdelta,lb_packetsreceiveddelta,lb_packetssentdelta,lb_connecteddevicescount,ub_id,ub_networkdeviceid,ub_eventdate,ub_eventtime,ub_bssid,ub_channel,ub_channelsinuse,ub_transmitpower,ub_min_rssi_per_cpe,ub_max_rssi_per_cpe,ub_avg_rssi_per_cpe,ub_perc5_rssi_per_cpe,ub_perc25_rssi_per_cpe,ub_perc50_rssi_per_cpe,ub_perc75_rssi_per_cpe,ub_perc95_rssi_per_cpe,ub_bytesreceived,ub_bytessent,ub_errorsreceived,ub_errorssent,ub_packetsreceived,ub_packetssent,ub_bytesreceiveddelta,ub_bytessentdelta,ub_errorsreceiveddelta,ub_errorssentdelta,ub_packetsreceiveddelta,ub_packetssentdelta,ub_connecteddevicescount,label_internet_not_working,label_optical_port,label_reset_not_work,label_sim_slot,label_no_problem,label_healthy,label_reset_itself,label_wifi,label_significant_physical_damage_irreparable,label_lan_port,label_telefon_port,label_blocked,label_device_overheating,label_poor_speed,label_does_not_log_into_acs,label_software,label_dsl_port,label_minor_physical_thermal_damage,label_power_supply
0,100002534,J912BH005997,45626428.0,100002534.0,Neispravna oprema - 33,2020-08-04 1

In [8]:
df.select('assetid').distinct().count()

33832

# 2. Config Preprocessing

In [9]:
config_preprocess = {
        'features_drop': 
                [ 'CPE_SERIAL_NUMBER','TICKET_ID','ASSET_ID',
                'TICKET_TENTATIVE_ROOT_CAUSE','START_DATE_TICKET',
                'DATE_SCAN_START','DATE_SCAN_END','CPE_ISSUE',
                'WINDOW_END','WINDOW_START', 'dslam_port_inst_id',
                'dslam_rg_port_inst_id','dslam_prim_izvod_inst_id',
                'dslam_date_inserted','dslam_bandwidth','dslam_vendor',
                'dslam_regija', 'acs_@type','acs_crm_accessNetwork', 
                'acs_crm_dslamFrame', 'acs_crm_dslamSlot','acs_xdslline_additionalSoftwareVersion', 
                'acs_xdslline_anexType','acs_xdslline_linkEncapsulationType',
                'acs_xdslline_pppoEUsername','acs_createdmsec','acs_xdslline_lossOfPower',
                'acs_xdslline_manufacturer', 'acs_xdslline_manufacturerOui', 'acs_xdslline_softwareVersion'],

        'features_uptime': 
                ['acs_xdslline_crc','acs_xdslline_fec','acs_xdslline_hec',
                'acs_xdslline_erroredSecs','acs_xdslline_cellDelin',
                'acs_xdslline_erroredSecs', 'acs_xdslline_initErrors',
                'acs_xdslline_initTimeOuts','acs_xdslline_severelyErroredSecs',
                'acs_xdslline_linkRetrain','acs_xdslline_lossOfFraming',
                'acs_xdslline_transmitBlocks','acs_xdslline_receiveBlocks', 
                'acs_xdslline_atuccrc','acs_xdslline_atucfec','acs_xdslline_atuchec',
                'lb_bytesreceived', 'lb_bytessent', 'lb_errorsreceived', 'lb_errorssent', 'lb_packetsreceived','lb_packetssent',
                'ub_bytesreceived', 'ub_bytessent', 'ub_errorsreceived', 'ub_errorssent', 'ub_packetsreceived','ub_packetssent',],

        'features_categorical': 
                ['dslam_platforma', 'dslam_model', 'dslam_card_type','dslam_vdsl_adsl',                                             
                'dslam_internet', 'dslam_iptv', 'dslam_voip',                                               
                'acs_crm_serviceName', 'cpetype'],
    
        'features_numerical': ['dslam_distinct_modulation',
                 'dslam_no_of_dominant_modulation',
                 'dslam_code_of_dominant_modulation',
                 'dslam_avg_bitrate_us',
                 'dslam_avg_bitrate_ds',
                 'dslam_avg_attenuation_ds',
                 'dslam_avg_attenuation_us',
                 'dslam_avg_power_us',
                 'dslam_avg_power_ds',
                 'dslam_avg_att_bitrate_us',
                 'dslam_avg_att_bitrate_ds',
                 'dslam_avg_snr_us',
                 'dslam_avg_snr_ds',
                 'dslam_no_counts',
                 'dslam_avg_bandline_ds',
                 'dslam_avg_bandline_us',
                 'dslam_sum_cv_us',
                 'dslam_count_cv_us',
                 'dslam_sum_cv_ds',
                 'dslam_count_cv_ds',
                 'dslam_sum_es_ds',
                 'dslam_count_es_ds',
                 'dslam_sum_es_us',
                 'dslam_count_es_us',
                 'dslam_sum_ses_ds',
                 'dslam_count_ses_ds',
                 'dslam_sum_ses_us',
                 'dslam_count_ses_us',
                 'dslam_sum_fec_ds',
                 'dslam_count_fec_ds',
                 'dslam_sum_fec_us',
                 'dslam_count_fec_us',
                 'dslam_inits',
                 'snmp_no_lossoflink',
                 'snmp_no_linkup',
                 'snmp_no_dyinggasp',
                 'acs_xdslline_dslUptime',
                 'acs_xdslline_cpeUptime',
                 'acs_xdslline_cpuUsagePercentage',
                 'acs_xdslline_freeMemoryKb',
                 'acs_xdslline_totalMemoryKb',
                 'acs_xdslline_usedMemoryKb',
                 'acs_xdslline_natCurrent',
                 'acs_xdslline_natMax',
                 'acs_xdslline_crc',
                 'acs_xdslline_fec',
                 'acs_xdslline_hec',
                 'acs_xdslline_cellDelin',
                 'acs_xdslline_erroredSecs',
                 'acs_xdslline_initErrors',
                 'acs_xdslline_initTimeOuts',
                 'acs_xdslline_severelyErroredSecs',
                 'acs_xdslline_linkRetrain',
                 'acs_xdslline_lossOfFraming',
                 'acs_xdslline_transmitBlocks',
                 'acs_xdslline_receiveBlocks',
                 'acs_xdslline_downAttenuationDb',
                 'acs_xdslline_downCurrRateKbps',
                 'acs_xdslline_downMaxRateKbps',
                 'acs_xdslline_downNoiseMarginDb',
                 'acs_xdslline_downPowerDbm',
                 'acs_xdslline_upAttenuationDb',
                 'acs_xdslline_upCurrRateKbps',
                 'acs_xdslline_upMaxRateKbps',
                 'acs_xdslline_upNoiseMarginDb',
                 'acs_xdslline_upPowerDbm',
                 'acs_xdslline_atuccrc',
                 'acs_xdslline_atucfec',
                 'acs_xdslline_atuchec',
                 'acs_cpe_reset_last_day',
                 'acs_xdslline_usedMemoryPercent',
                 'acs_xdslline_natCurrentPercent',
                 'acs_xdslline_upCurrRateDiff',
                 'acs_xdslline_downCurrRateDiff',
                 'lb_bytesreceived', 'lb_bytessent', 'lb_errorsreceived', 'lb_errorssent', 'lb_packetsreceived','lb_packetssent',
                 'lb_connecteddevicescount', 'lb_transmitpower','lb_min_rssi_per_cpe','lb_max_rssi_per_cpe','lb_avg_rssi_per_cpe',
                 'lb_errorsreceiveddelta','lb_errorssentdelta','lb_packetsreceiveddelta','lb_packetssentdelta',
                 'ub_bytesreceived', 'ub_bytessent', 'ub_errorsreceived', 'ub_errorssent', 'ub_packetsreceived','ub_packetssent',
                 'ub_connecteddevicescount', 'ub_transmitpower','ub_min_rssi_per_cpe','ub_max_rssi_per_cpe','ub_avg_rssi_per_cpe',
                 'ub_errorsreceiveddelta','ub_errorssentdelta','ub_packetsreceiveddelta','ub_packetssentdelta'],

        'labels': 
                ['label_cpe_replacement',
                'label_optical_port',
                'label_reset_not_work',
                'label_sim_slot',
                'label_no_problem',
                'label_healthy',
                'label_reset_itself',
                'label_wifi',
                'label_significant_physical_damage_irreparable',
                'label_lan_port',
                'label_telefon_port',
                'label_blocked',
                'label_device_overheating',
                'label_poor_speed',
                'label_does_not_log_into_acs',
                'label_software',
                'label_dsl_port',
                'label_minor_physical_thermal_damage',
                'label_power_supply',
                'label_internet_not_working',
                ],
    'scaling_type': 'maxmin'

    
}


# 3. Preprocessing

In [10]:
# Load config
features_drop = config_preprocess['features_drop']
features_uptime = config_preprocess['features_uptime']
labels = config_preprocess['labels']
features_categorical = config_preprocess['features_categorical']
features_numerical = config_preprocess['features_numerical']
scaling_type = config_preprocess['scaling_type']

## 3.1 Initial Filtering per valid samples

In [11]:
df = filter_per_valid_samples(df, count_valid_samples=25)

## 3.2 Categorical preprocessing

Filling missing FIX categoried

In [12]:
df_inventory = get_fix_categories(df, config_preprocess['features_categorical'])
df_inventory, service_type_list = spit_sevice(df_inventory)
features_get_dummies = [x for x in features_categorical if x not in ['acs_crm_serviceName']]
df_categorical, features_categorical_onehot = spark_get_dummies_fix(df_inventory, features_get_dummies, service_type_list)

Filling categorical fearture: dslam_platforma
Filling categorical fearture: dslam_model
Filling categorical fearture: dslam_card_type
Filling categorical fearture: dslam_vdsl_adsl
Filling categorical fearture: dslam_internet
Filling categorical fearture: dslam_iptv
Filling categorical fearture: dslam_voip
Filling categorical fearture: acs_crm_serviceName
Filling categorical fearture: cpetype


In [13]:
df_categorical.limit(5).toPandas()

,dslam_platforma_ERICSSONEDA,dslam_platforma_HUAWEISMARTAX,dslam_model_MA5603T,dslam_model_MA5608T,dslam_model_EDAMEDIUMSUBRACK,dslam_model_EDAVDSLSUBRACK,dslam_model_MA5622A,dslam_model_MA5623A,dslam_model_MA5600T,dslam_model_EDASMALLSUBRACK,dslam_card_type_VDPM,dslam_card_type_ADQD,dslam_card_type_BCVSE,dslam_card_type_BCVSB,dslam_card_type_EDN612I,dslam_card_type_VDNF,dslam_card_type_VCPM,dslam_card_type_EDN312XI,dslam_card_type_SDPM,dslam_card_type_ADIF,dslam_card_type_BVCMM,dslam_vdsl_adsl_ADSL,dslam_vdsl_adsl_VDSL,dslam_internet_Y,dslam_internet_N,dslam_iptv_Y,dslam_iptv_N,dslam_voip_Y,dslam_voip_N,cpetype_HA35-11,cpetype_Smart3,cpetype_SpeedportPlus,cpetype_ZXDSL931VII,cpetype_W724Ci,cpetype_SpeedportPlus181,cpetype_ZXDSL931VIIV4.0,cpetype_FRITZ!Box_6890,cpetype_W724Ci_11AC,ADSLInternet,DTH,ResidentVoIP,ADSLSMART,MultiOffice,IPTV/DTH,PMUP,ADSLMultiOffice,BusinessVoIP,IPTV,IPTVSolo,ADSLBusinessVoIPSolo,Halo3,ADSLPMUP,ADSLCPETest,ResidentVoIPSolo,assetid
0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,100097226
1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,100477483
2,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,100613042
3,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,100730500
4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,101523903


In [14]:
len(features_categorical_onehot)

54

## 3.3 Numerical Preprocessing 

In [15]:
# Hand Craft Features
df = hand_craft_features(df)
# Dropping features that are not required
df = df.drop(*features_drop)
# Features scale by Uptime
df = scale_by_uptime(df, features_uptime)
# Scaling Numerical Features
df =  apply_scaling(df, scaling_type, features_numerical)

Applying Max-Min Scaling


## 3.4 Join Numerical and categorical

In [16]:
# Selecting required no categorical features and join with one hot encoding
df = df.select(*['assetid', 'datum'] + features_numerical + labels) \
       .join(df_categorical, on=['assetid'], how='left')
# Filter by valid sequence length 
df = filter_by_valid_seq_len(df, seq_len=32)
# Filling Missing values
df = df.fillna(-3.0)
# Fixing Names
df, features_selected = column_name_fix(df, features_numerical + features_categorical_onehot)

In [17]:
df.select('assetid').distinct().count()

20125

In [29]:
columns = df.columns
labels = [x for x in columns if 'label' in x]
no_categorical = labels + features_numerical + ['assetid', 'datum']
features_categorical_onehot = [x for x in columns if x not in no_categorical]

print('\n' * 3 + '-' * 75 + '\n NUMERICAL FEATURES SELECTED ' + '\n' + '-' * 75  )
print('The following list of features need to be updated on configs.preprocessing.feature_selected \n')
print('Number of Numerical Features:',len(features_numerical), '\n')
print(features_numerical)


print('\n' * 3 + '-' * 75 + '\n CATEGORICAL FEATURES SELECTED ' + '\n' + '-' * 75 )
print('The following list of features need to be updated on configs.preprocessing.feature_selected \n')
print('Number of Numerical Features:',len(features_categorical_onehot), '\n')
print(features_categorical_onehot)





---------------------------------------------------------------------------
 NUMERICAL FEATURES SELECTED 
---------------------------------------------------------------------------
The following list of features need to be updated on configs.preprocessing.feature_selected 

Number of Numerical Features: 104 

['dslam_distinct_modulation', 'dslam_no_of_dominant_modulation', 'dslam_code_of_dominant_modulation', 'dslam_avg_bitrate_us', 'dslam_avg_bitrate_ds', 'dslam_avg_attenuation_ds', 'dslam_avg_attenuation_us', 'dslam_avg_power_us', 'dslam_avg_power_ds', 'dslam_avg_att_bitrate_us', 'dslam_avg_att_bitrate_ds', 'dslam_avg_snr_us', 'dslam_avg_snr_ds', 'dslam_no_counts', 'dslam_avg_bandline_ds', 'dslam_avg_bandline_us', 'dslam_sum_cv_us', 'dslam_count_cv_us', 'dslam_sum_cv_ds', 'dslam_count_cv_ds', 'dslam_sum_es_ds', 'dslam_count_es_ds', 'dslam_sum_es_us', 'dslam_count_es_us', 'dslam_sum_ses_ds', 'dslam_count_ses_ds', 'dslam_sum_ses_us', 'dslam_count_ses_us', 'dslam_sum_fec_ds', 'dslam

## 3.5 Split Train and Test

In [19]:
# Split Train
df_train = df_train.filter(F.col('cpetype').isin(cpetypes)).select('assetid', 'datum')
df_train = df.join(df_train, on=['assetid', 'datum'], how='inner')
df_train = df_train.select(['assetid', 'datum'] + features_selected + labels)

In [20]:
# Split Test
df_test = df_test.filter(F.col('cpetype').isin(cpetypes)).select('assetid', 'datum')
df_test = df.join(df_test, on=['assetid', 'datum'], how='inner')
df_test = df_test.select(['assetid', 'datum'] + features_selected + labels)

In [34]:
# Saving preprocessed test and train dataset
df_train.write.mode('overwrite').parquet(path_data_preprocessed + "/"+str(scaling_type)+"/train.parquet")
df_test.write.mode('overwrite').parquet(path_data_preprocessed + "/"+str(scaling_type)+"/test.parquet")

In [35]:
print('Number numerical:',len(features_numerical))
print('Number categorical:',len(features_categorical_onehot))
print('Total number of features:',len(features_categorical_onehot)+len(features_numerical))

Number numerical: 104
Number categorical: 54
Total number of features: 158


# Appendix

## A. Functions

In [3]:


import json
import sys
import os
import itertools

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable



def scale_by_uptime(df, feature_list):
    for feature in feature_list:
        df = df.withColumn(feature, F.col(feature) / F.col('acs_xdslline_cpeUptime'))
    return df


def fix_cpetype(df):
    df_inventory = df.filter(F.col('cpetype').isNotNull()).select('assetid', 'cpetype')\
                     .groupBy('assetid', 'cpetype').count()
    df_inventory = df_inventory.select('assetid', 'cpetype')
    df_inventory = df_inventory.dropDuplicates(subset=['assetid'])
    df = df.drop('cpetype')
    df = df.join(df_inventory, on=['assetid'], how='left')
    df = df.filter(F.col('cpetype').isNotNull())
    df = df.filter((F.col('cpetype') != 'FRITZ!Box_7590'))
    df = df.filter((F.col('cpetype') != 'FRITZ!Box_6890'))
    return df


def filter_per_valid_samples(df, count_valid_samples):
    # Filtering minimin numbre of missing values for DSLAM
    df_count_valid_dslam = df.select('assetid','dslam_model')
    df_count_valid_dslam = df_count_valid_dslam.na.drop(subset=['dslam_model'])
    df_count_valid_dslam = df_count_valid_dslam.na.drop(subset=['assetid'])
    df_count_valid_dslam = df_count_valid_dslam.groupBy('assetid','dslam_model').count()
    df_count_valid_dslam_filter = df_count_valid_dslam.filter(F.col('count')>=count_valid_samples)
    df_count_valid_dslam_filter = df_count_valid_dslam_filter.dropDuplicates(['assetid'])
    df = df.join(df_count_valid_dslam_filter.select('assetid'), on=['assetid'], how='right')
    return df

def filter_by_valid_seq_len(df, seq_len):
    # Filtering by valid sequence length
    df_count_valid_dslam = df.select('assetid','dslam_platforma_ERICSSONEDA')
    df_count_valid_dslam = df_count_valid_dslam.na.drop(subset=['dslam_platforma_ERICSSONEDA'])
    df_count_valid_dslam = df_count_valid_dslam.na.drop(subset=['assetid'])
    df_count_valid_dslam = df_count_valid_dslam.groupBy('assetid','dslam_platforma_ERICSSONEDA').count()
    df_count_valid_dslam_filter = df_count_valid_dslam.filter(F.col('count')==seq_len)
    df_count_valid_dslam_filter = df_count_valid_dslam_filter.dropDuplicates(['assetid'])
    df = df.join(df_count_valid_dslam_filter.select('assetid'), on=['assetid'], how='right')
    return df

def filter_fix_categories(df, feature):
    df = df.select('assetid', feature).distinct()
    df = df.na.drop()
    df = df.dropDuplicates(['assetid'])
    return df


def get_fix_categories(df, features):
    df_inventory=df.select('assetid').distinct()
    df_inventory.write.mode('overwrite').parquet(path_data_interim + "/inventory0.parquet")
    for i, feature in enumerate(features):
        print('Filling categorical fearture:',feature)
        df_category = filter_fix_categories(df, feature)
        df_inventory = spark.read.parquet(path_data_interim + "/inventory"+str(i)+".parquet")
        df_inventory= df_inventory.join(df_category, on =['assetid'], how='left')
        df_inventory.write.mode('overwrite').parquet(path_data_interim + "/inventory"+str(i+1)+".parquet")
    df_inventory = df_inventory.dropna()
    return df_inventory


def spit_sevice(df_inventory):
    service_types = df_inventory.select('acs_crm_serviceName').distinct().toPandas()
    service_type_list = []
    for service_type in service_types['acs_crm_serviceName'].values:
        service_type_sublist = str(service_type).split(' + ')
        for element in service_type_sublist:
            if element not in service_type_list:
                service_type_list.append(element)
    for element in service_type_list:
        df_inventory = df_inventory.withColumn(element, F.when(F.col('acs_crm_serviceName').contains(element),1).otherwise(0))
    df_inventory = df_inventory.drop('acs_crm_serviceName')
    return df_inventory, service_type_list


def hand_craft_features(df):
    df = df.withColumn('acs_cpe_reset_last_day', \
                       F.when(F.col('acs_xdslline_cpeUptime') < 86400, 1.0).otherwise(0.0))

    df = df.withColumn('acs_xdslline_usedMemoryPercent', \
                       F.col('acs_xdslline_usedMemoryKb') / F.col('acs_xdslline_totalMemoryKb'))

    df = df.withColumn('acs_xdslline_natCurrentPercent', \
                       F.col('acs_xdslline_natCurrent') / F.col('acs_xdslline_natMax'))

    df = df.withColumn('acs_xdslline_upCurrRateDiff', \
                       F.col('acs_xdslline_upMaxRateKbps') \
                       - F.col('acs_xdslline_upCurrRateKbps'))

    df = df.withColumn('acs_xdslline_downCurrRateDiff', \
                       F.col('acs_xdslline_downMaxRateKbps') \
                       - F.col('acs_xdslline_downCurrRateKbps'))

    df = df.withColumn('label_cpe_replacement', \
                       F.when((F.col('label_healthy') == 0) & \
                              (F.col('label_no_problem') == 0), 1).otherwise(0))
    return df


def spark_get_dummies(df, features_categorical):
    categories = []
    for i, values in enumerate(features_categorical):
        categories.append(df.select(values).distinct().rdd.flatMap(lambda x: x).collect())

    expressions = []
    for i, values in enumerate(features_categorical):
        expressions.append([F.when(F.col(values) == i, 1).otherwise(0).alias(str(values) + "_" + str(i)) for i in categories[i]])

    expressions_flat = list(itertools.chain.from_iterable(expressions))
    expressions_flat.append('assetid')
    expressions_flat.append('datum')
    df_final = df.select(*expressions_flat)

    for feature in df_final.columns:
        if len(str(feature)) != len(str(feature).replace(' ', '')):
            df_final = df_final.withColumnRenamed(feature, feature.replace(' ', ''))
    features_categorical_new = [feature for feature in df_final.columns]
    [features_categorical_new.remove(feature) for feature in ['assetid', 'datum']]
    return df_final, features_categorical_new

def spark_get_dummies_fix(df, features_categorical, features_binary):
    categories = []
    for i, values in enumerate(features_categorical):
        categories.append(df.select(values).distinct().rdd.flatMap(lambda x: x).collect())

    expressions = []
    for i, values in enumerate(features_categorical):
        expressions.append([F.when(F.col(values) == i, 1).otherwise(0).alias(str(values) + "_" + str(i)) for i in categories[i]])

    expressions_flat = list(itertools.chain.from_iterable(expressions))
    features_extra = features_binary + ['assetid']
    for feature_extra in features_extra:
        expressions_flat.append(feature_extra)
    df_final = df.select(*expressions_flat)

    for feature in df_final.columns:
        if len(str(feature)) != len(str(feature).replace(' ', '')):
            df_final = df_final.withColumnRenamed(feature, feature.replace(' ', ''))
    features_categorical_new= [feature for feature in df_final.columns if feature not in ['assetid']]
    return df_final, features_categorical_new

def standar_scaler(df, features):
    feature_mean = dict()
    feature_stddev = dict()
    df_stats = df.select(*features).describe()
    for feature in features:
        try:
            feature_mean[feature] = float(df_stats.filter(F.col('summary')=='mean').select(feature).head()[0])
            feature_stddev[feature] = float(df_stats.filter(F.col('summary')=='stddev').select(feature).head()[0])
        except:
            feature_mean[feature] = 0.0
            feature_stddev[feature] = 1.0
    return feature_mean, feature_stddev

def apply_standar_scaling(df, feature_mean, feature_stddev):
    for feature, val_mean in feature_mean.items():
        val_stddev = feature_stddev[feature]
        df = df.withColumn(feature, (F.col(feature) - val_mean) / val_stddev)
    return df

def maxmin_scaler(df,features):
    feature_max = dict()
    feature_min = dict()
    df_min_max = df.select(*features).describe()
    for feature in features:
        try:
            feature_max[feature] = float(df_min_max.filter(F.col('summary')=='max').select(feature).head()[0])
            feature_min[feature] = float(df_min_max.filter(F.col('summary')=='min').select(feature).head()[0])
        except:
            feature_max[feature] = 1.0
            feature_min[feature] = 0.0
    return feature_min, feature_max



def apply_maxmin_scaling(df, feature_min, feature_max):
    for feature, val_min in feature_min.items():
        val_max = feature_max[feature]
        df = df.withColumn(feature, (F.col(feature) - val_min) / (val_max - val_min))
    return df

def apply_scaling(df, scaling_type, features_numerical):
    if scaling_type == 'standard':
        print('Applying Standard Scaling')
        feature_mean, feature_stddev = standar_scaler(df, features_numerical)
        df_scale = apply_standar_scaling(df, feature_mean, feature_stddev)
    if scaling_type == 'maxmin':
        print('Applying Max-Min Scaling')
        feature_min, feature_max = maxmin_scaler(df,features_numerical)
        df_scale = apply_maxmin_scaling(df, feature_min, feature_max)
    return df_scale


def save_scaler(feature_min, feature_max, path_data_preprocessed):
    path_scaler = path_data_preprocessed + "/scaler"
    if not os.path.exists(path_scaler):
        os.makedirs(path_scaler)
    json.dump(feature_min, open(path_scaler + "/feature_min.json", 'w'))
    json.dump(feature_max, open(path_scaler + "/feature_max.json", 'w'))


def spark_init():
    """Initialization of the Spark Session

    Returns:
        spark (object): Spark Session.
    """
    spark = SparkSession.builder \
                        .appName("cpe-replacement") \
                        .getOrCreate()
    return spark


def column_name_fix(df, features):
    substring_list = ['.', '(', ')']
    new_features = list()
    new_columns = list()
    for column in df.columns:
        if any(substring in column for substring in substring_list):
            new_name = column
            for substring in substring_list:
                new_name = new_name.replace(substring, '')
            new_columns.append(new_name)
        else:
            new_columns.append(column)
    
    for new_feature in features:
        for substring in substring_list:
            new_feature = new_feature.replace(substring, '')
        new_features.append(new_feature)
    df = df.toDF(*new_columns)
    return df, new_features



